<a href="https://colab.research.google.com/github/mr-nastik/BCI-Competition-EEG-signal-Classification/blob/main/BCI_competion_IV_dataset_2a_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
pip install -U mne

Requirement already up-to-date: mne in /usr/local/lib/python3.6/dist-packages (0.20.7)


In [ ]:
import os
import mne
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_gdf
from mne.decoding import CSP

from scipy import linalg

import scipy.io as sio

from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.multiclass import OneVsOneClassifier
from sklearn.feature_selection import SelectKBest,mutual_info_classif

from tqdm import tqdm

In [ ]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
os.chdir('/content/gdrive/My Drive/summer_intern/')

In [ ]:
raw_names=['A0'+str(i)+'T.gdf' for i in range(1,10)]
raw=concatenate_raws([read_raw_gdf(i,preload=True,verbose=False) for i in raw_names])

In [ ]:
raw

<RawGDF | A01T.gdf, 25 x 6005911 (24023.6 s), ~1.12 GB, data loaded>

In [ ]:
tmn,tmx=0.5,3.0
event_id=dict({'769':0,'770':1,'771':2,'772':3})
event, _ =events_from_annotations(raw, event_id=event_id)
raw.info['bads']+=['EOG-left','EOG-right','EOG-central']
picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,exclude='bads')
print(picks)

Used Annotations descriptions: ['769', '770', '771', '772']
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]


In [ ]:
filters=np.array([[4,8],[8,12],[12,16],[16,20],[20,24],[24,28],[28,32],[32,36],[36,40]])

In [ ]:
epochs=[]
i=0
for l,r in filters:
    epochs+=[Epochs(raw, event, event_id,tmin=tmn,tmax=tmx, proj=True, picks=picks,baseline=None, preload=True)]
    epochs[i].filter(l,r,verbose=False)
    i+=1
labels = epochs[0].events[:, -1]
print(labels)
num_trail=len(labels)

2592 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 2592 events and 626 original time points ...
0 bad epochs dropped
2592 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 2592 events and 626 original time points ...
0 bad epochs dropped
2592 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 2592 events and 626 original time points ...
0 bad epochs dropped
2592 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 2592 events and 626 original time points ...
0 bad epochs dropped
2592 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 2592 events and 626 original time points ...
0 bad epochs dropped
2592 matching events found
No baseline correc

# 10k fold validation

In [ ]:
acc = []
ka = []

cv = ShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 0)
for train_idx, test_idx in cv.split(labels):
    
    y_train, y_test = labels[train_idx], labels[test_idx]
    
    csp=[]
    x_train=np.zeros((len(y_train)))
    #print('x_train shape: ',x_train.shape)
    x_test=np.zeros((len(y_test)))
    #print('x_train shape: ',x_test.shape)

    i=0

    csp=[]
    x_train=np.zeros((len(y_train)))

    i=0

    for l,r in filters:
        csp+=[CSP(n_components=32, reg=None, log=True, norm_trace=True)]
        data=epochs[i].get_data()
        temp=csp[i].fit_transform(data[train_idx,:,:],y_train)
        temp1=csp[i].transform(data[test_idx,:,:])
        if (i==0):
            x_train=temp
            x_test=temp1
        else:
            x_train=np.concatenate((x_train,temp),axis=-1)
            x_test=np.concatenate((x_test,temp1),axis=-1)
        i+=1

    feat_selection=SelectKBest(mutual_info_classif,k=44)
    x_train=feat_selection.fit_transform(x_train,y_train)
    x_test=feat_selection.transform(x_test)

    normalize=preprocessing.StandardScaler()
    x_train=normalize.fit_transform(x_train)
    x_test=normalize.transform(x_test)

    M = svm.SVC(3.45)
    model=OneVsOneClassifier(M)
    model.fit(x_train, y_train)
    pred = model.predict(x_test)

    acc.append(accuracy_score(y_test, pred))
    ka.append(cohen_kappa_score(y_test, pred))


Computing rank from data with rank=None
    Using tolerance 5.6e-05 (2.2e-16 eps * 22 dim * 1.2e+10  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.5e-05 (2.2e-16 eps * 22 dim * 1.1e+10  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 5.7e-05 (2.2e-16 eps * 22 dim * 1.2e+10  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6e-05 (2.2e-16 eps * 22 dim * 1.2e+10  max singular value)
  

In [ ]:
scores = {'Accuracy':acc,'Kappa':ka}
Es = pd.DataFrame(scores)
avg = {'Accuracy':[np.mean(acc)],'Kappa':[np.mean(ka)]}
Avg = pd.DataFrame(avg)
T = pd.concat([Es,Avg])
T.index = ['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7','Fold8','Fold9','Fold10','Avg']
T.index.rename('Fold',inplace=True)
print(T)


        Accuracy     Kappa
Fold                      
Fold1   0.666667  0.555406
Fold2   0.684008  0.577531
Fold3   0.724470  0.632438
Fold4   0.689788  0.586591
Fold5   0.691715  0.588556
Fold6   0.672447  0.564105
Fold7   0.710983  0.614270
Fold8   0.676301  0.567476
Fold9   0.668593  0.558208
Fold10  0.707129  0.609628
Avg     0.689210  0.585421


# testing on evalution file

In [ ]:

tr=np.arange(len(labels))
np.random.shuffle(tr)
y_train=labels[tr]

csp=[]
x_train=np.zeros((len(y_train)))

i=0

for l,r in filters:
    csp+=[CSP(n_components=32, reg=None, log=True, norm_trace=True)]
    data=epochs[i].get_data()
    temp=csp[i].fit_transform(data[tr,:,:],y_train)
    if (i==0):
        x_train=temp
    else:
        x_train=np.concatenate((x_train,temp),axis=-1)
        
    i+=1

feat_selection=SelectKBest(mutual_info_classif,k=44)
x_train=feat_selection.fit_transform(x_train,y_train)


normalize = preprocessing.StandardScaler()
x_train = normalize.fit_transform(x_train)

M = svm.SVC(3.45)
model=OneVsOneClassifier(M)
model.fit(x_train, y_train)


Computing rank from data with rank=None
    Using tolerance 6.3e-05 (2.2e-16 eps * 22 dim * 1.3e+10  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.2e-05 (2.2e-16 eps * 22 dim * 1.3e+10  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.4e-05 (2.2e-16 eps * 22 dim * 1.3e+10  max singular value)
    Estimated rank (mag): 22
    MAG: rank 22 computed from 22 data channels with 0 projectors
Reducing data rank from 22 -> 22
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.7e-05 (2.2e-16 eps * 22 dim * 1.4e+10  max singular value)


OneVsOneClassifier(estimator=SVC(C=3.45, break_ties=False, cache_size=200,
                                 class_weight=None, coef0=0.0,
                                 decision_function_shape='ovr', degree=3,
                                 gamma='scale', kernel='rbf', max_iter=-1,
                                 probability=False, random_state=None,
                                 shrinking=True, tol=0.001, verbose=False),
                   n_jobs=None)

In [ ]:
kappa=[]
accuracy=[]
for z in range(1,10):
    y_test=sio.loadmat('true_labels/A0'+str(z)+'E.mat')
    y_test=y_test['classlabel'].reshape(-1)
    y_test=y_test-1

    raw1 = mne.io.read_raw_gdf("A0"+str(z)+"E.gdf", preload=True,verbose=False)
    
    events=mne.events_from_annotations(raw1,verbose=False)

    t=len(events[1])
    event_ids={"cue": t}
    tepochs=[]
    i=0
    for l,r in filters:
        tepochs+=[Epochs(raw1, events[0], event_ids,tmin=tmn,tmax=tmx, proj=True, picks=picks,baseline=None, preload=True,verbose=False)]
        tepochs[i].filter(l,r,verbose=False)
        i+=1
    x_test=np.zeros((len(y_test)))

    i=0

    for l,r in filters:
        data1=tepochs[i].get_data()
        temp1=csp[i].transform(data1)
        if (i==0):
            x_test=temp1
        else:
            x_test=np.concatenate((x_test,temp1),axis=-1)
            
        i+=1
    
    x_test=feat_selection.transform(x_test)
    

    x_test=normalize.transform(x_test)
    y_pred=model.predict(x_test)
    accuracy+=[accuracy_score(y_test,y_pred)]
    kappa+=[cohen_kappa_score(y_test,y_pred)]
    
    print('SUBJECT'+str(z)+':-')
    print('accuracy of subject'+str(z)+': \t',accuracy_score(y_test,y_pred))
    print('kappa of subject'+str(z)+': \t',cohen_kappa_score(y_test,y_pred))
    print('Confusion Matrix :-')
    print(confusion_matrix(y_test,y_pred))
    print('\n\n')

SUBJECT1:-
accuracy of subject1: 	 0.8333333333333334
kappa of subject1: 	 0.7777777777777778
Confusion Matrix :-
[[70  1  1  0]
 [11 58  3  0]
 [ 4  2 63  3]
 [ 5  2 16 49]]



SUBJECT2:-
accuracy of subject2: 	 0.5069444444444444
kappa of subject2: 	 0.34259259259259256
Confusion Matrix :-
[[31 20 12  9]
 [23 35 10  4]
 [ 7  8 56  1]
 [16 16 16 24]]



SUBJECT3:-
accuracy of subject3: 	 0.8090277777777778
kappa of subject3: 	 0.7453703703703703
Confusion Matrix :-
[[58  7  2  5]
 [ 0 70  1  1]
 [ 4  9 44 15]
 [ 3  7  1 61]]



SUBJECT4:-
accuracy of subject4: 	 0.5208333333333334
kappa of subject4: 	 0.36111111111111116
Confusion Matrix :-
[[ 9 57  5  1]
 [ 3 64  3  2]
 [ 4 13 26 29]
 [ 1 15  5 51]]



SUBJECT5:-
accuracy of subject5: 	 0.3854166666666667
kappa of subject5: 	 0.18055555555555558
Confusion Matrix :-
[[38 33  1  0]
 [ 5 67  0  0]
 [20 48  4  0]
 [29 33  8  2]]



SUBJECT6:-
accuracy of subject6: 	 0.4826388888888889
kappa of subject6: 	 0.31018518518518523
Confusion Ma

In [ ]:
scores = {'Accuracy':accuracy,'Kappa':kappa}
Es = pd.DataFrame(scores)
avg = {'Accuracy':[np.mean(accuracy)],'Kappa':[np.mean(kappa)]}
Avg = pd.DataFrame(avg)
T = pd.concat([Es,Avg])
T.index = ['A01','A02','A03','A04','A05','A06','A07','A08','A09','Avg']
T.index.rename('Subject',inplace=True)
score1=np.mean(kappa)
print(T)

         Accuracy     Kappa
Subject                    
A01      0.833333  0.777778
A02      0.506944  0.342593
A03      0.809028  0.745370
A04      0.520833  0.361111
A05      0.385417  0.180556
A06      0.482639  0.310185
A07      0.756944  0.675926
A08      0.736111  0.648148
A09      0.718750  0.625000
Avg      0.638889  0.518519


# subjectwise model

In [ ]:
kappa=[]
accuracy=[]



for z in range(1,10):

    raw=read_raw_gdf('A0'+str(z)+'T.gdf',preload=True,verbose=False)


    tmn,tmx=0.5,3.0
    event_id=dict({'769':0,'770':1,'771':2,'772':3})
    event, _ =events_from_annotations(raw, event_id=event_id)
    raw.info['bads']+=['EOG-left','EOG-right','EOG-central']
    picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False,exclude='bads')

    epochs=[]
    i=0
    for l,r in filters:
        epochs+=[Epochs(raw, event, event_id,tmin=tmn,tmax=tmx, proj=True, picks=picks,baseline=None, preload=True)]
        epochs[i].filter(l,r,verbose=False)
        i+=1
    labels = epochs[0].events[:, -1]






    tr=np.arange(len(labels))
    np.random.shuffle(tr)
    y_train=labels[tr]

    csp=[]
    x_train=np.zeros((len(y_train)))

    i=0

    for l,r in filters:
        csp+=[CSP(n_components=32, reg=None, log=True, norm_trace=True)]
        data=epochs[i].get_data()
        temp=csp[i].fit_transform(data[tr,:,:],y_train)
        if (i==0):
            x_train=temp
        else:
            x_train=np.concatenate((x_train,temp),axis=-1)
            
        i+=1

    feat_selection=SelectKBest(mutual_info_classif,k=44)
    x_train=feat_selection.fit_transform(x_train,y_train)


    normalize = preprocessing.StandardScaler()
    x_train = normalize.fit_transform(x_train)

    M = svm.SVC(3.45)
    model=OneVsOneClassifier(M)
    model.fit(x_train, y_train)









    y_test=sio.loadmat('true_labels/A0'+str(z)+'E.mat')
    y_test=y_test['classlabel'].reshape(-1)
    y_test=y_test-1

    raw1 = mne.io.read_raw_gdf("A0"+str(z)+"E.gdf", preload=True,verbose=False)
    
    events=mne.events_from_annotations(raw1,verbose=False)

    t=len(events[1])
    event_ids={"cue": t}
    tepochs=[]
    i=0
    for l,r in filters:
        tepochs+=[Epochs(raw1, events[0], event_ids,tmin=tmn,tmax=tmx, proj=True, picks=picks,baseline=None, preload=True,verbose=False)]
        tepochs[i].filter(l,r,verbose=False)
        i+=1
    x_test=np.zeros((len(y_test)))

    i=0

    for l,r in filters:
        data1=tepochs[i].get_data()
        temp1=csp[i].transform(data1)
        if (i==0):
            x_test=temp1
        else:
            x_test=np.concatenate((x_test,temp1),axis=-1)
            
        i+=1
    
    x_test=feat_selection.transform(x_test)
    

    x_test=normalize.transform(x_test)
    y_pred=model.predict(x_test)
    accuracy+=[accuracy_score(y_test,y_pred)]
    kappa+=[cohen_kappa_score(y_test,y_pred)]
    
    print('SUBJECT'+str(z)+':-')
    print('accuracy of subject'+str(z)+': \t',accuracy_score(y_test,y_pred))
    print('kappa of subject'+str(z)+': \t',cohen_kappa_score(y_test,y_pred))
    print('Confusion Matrix :-')
    print(confusion_matrix(y_test,y_pred))
    print('\n\n')

Used Annotations descriptions: ['769', '770', '771', '772']
288 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 288 events and 626 original time points ...
0 bad epochs dropped
288 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 288 events and 626 original time points ...
0 bad epochs dropped
288 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 288 events and 626 original time points ...
0 bad epochs dropped
288 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 288 events and 626 original time points ...
0 bad epochs dropped
288 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 288 events and 626 original time points ...
0 bad epochs dro

In [ ]:
scores = {'Accuracy':accuracy,'Kappa':kappa}
Es = pd.DataFrame(scores)
avg = {'Accuracy':[np.mean(accuracy)],'Kappa':[np.mean(kappa)]}
Avg = pd.DataFrame(avg)
T = pd.concat([Es,Avg])
T.index = ['A01','A02','A03','A04','A05','A06','A07','A08','A09','Avg']
T.index.rename('Subject',inplace=True)
score2=np.mean(kappa)
print(T)

         Accuracy     Kappa
Subject                    
A01      0.815972  0.754630
A02      0.493056  0.324074
A03      0.784722  0.712963
A04      0.645833  0.527778
A05      0.527778  0.370370
A06      0.395833  0.194444
A07      0.847222  0.796296
A08      0.697917  0.597222
A09      0.725694  0.634259
Avg      0.659336  0.545782


# BCI Compition Score

In [ ]:
print('Score according to one model for all subjects: ',score1)
print('Score according to subjectwise model for all subjects: ',score2 )

Score according to one model for all subjects:  0.5185185185185185
Score according to subjectwise model for all subjects:  0.5457818930041154
